# 팀 프로젝트 (데이터 수집 프로젝트)

#### 데이터 분석 주제 선정

- 데이터를 분석하려는 목적 설정
- 목적에 부합하는 데이터를 제공하는 사이트 조사

#### 데이터 수집을 위한 설계

- 사이트로부터 어떤 데이터를 수집할 것인지 파악하고 수집 후 CSV파일로 저장하기 위한 설계
    + 단순 파일에 저장
    + 데이터 베이스를 활용하여 저장
    
#### 분석을 위한 데이터 수집

- 조사한 사이트들을 분담하여 Crawling 또는 Scraping을 활용하여 데이터 수집
- 수집한 데이터를 앞서 설계한 구조로 저장
- 저장된 데이터 확인

- 팀 프로젝트 (23.06.26 ~ 06.28)
 <br> 주제 : 국내 서점 온라인 사이트에서 베스트 셀러 가져오기
 <br> 수집할 자료 기간 : 22년 7월 ~ 23년 6월까지의 1년 간 베스트셀러 Top20위 자료 수집
 <br> 가져올 목록: ISBN(번호), 책 제목, 도서 장르, 저자, 평점, 판매가격, 발행일

<< 프로젝트 진행 과정 >>

데이터 수집할 사이트 : 교보문고

교보문고 사이트에서 베스트셀러 자료를 가져오기 위해 requests.get(url) 으로 자료를 가져오려고 했으나 
무슨 이유에서인지 사이트 점검으로 자료를 가져오지 못함. (하단의 출력된 사이트점검 메시지가 출력됨)
requests.post도 사용해봤지만 같은 서비스 점검 메시지 내용이 출력됨.

그래서 월 별 사이트의 url 페이지 소스를 가져와서 각 책에 해당하는 a 태그의 href 속성값을 출력.
각 책의 href 속성값으로 일일이 사이트를 방문하여 자료를 수집.

22.7월 부터 23.6월 까지 자료를 수집하려 했으나 교보문고는 23.5월까지의 자료밖에 없어서 22.7 ~ 23.5 까지 자료수집 진행.


< 자료 수집 과정 >
search_day()에서 각 월별 값을 출력. (22.07 ~ 23.05)
해당 값을 가지고 kyobo_book() 으로 값 전달. 월 별 페이지에서 책들의 링크를 추출하여 각각의 페이지에 접근.
사이트를 일일이 돌아다니다 보니 시간지연 등 오류가 자주 발생하고 중단되어 예외처리(try ~ except)활용하여 자료 수집.
각 페이지에서 ISBN, 제목, 장르 등등 자료를 수집하여 리스트에 값을 저장.
make_csv()를 통해 리스트에 저장된 값을 CSV로 저장.

In [3]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time, os


def kyobo_book(value):

    url = f"https://product.kyobobook.co.kr/bestseller/total?period=003#?page=1&per=20&period=003&ymw=20{value}&bsslBksClstCode=A"

    chrome = Chrome()
    chrome.get(url)

    time.sleep(3)
    soup = BeautifulSoup(chrome.page_source)

    time.sleep(1)
    chrome.close()

    kyobo_atag = soup.find_all('a', {'class':'prod_info'})
    
    try : 
        kyobo_list = []
        for url in kyobo_atag:

            kyobo_url = url.attrs['href']

            driver = Chrome()
            driver.get(kyobo_url)
            time.sleep(2)

            bs = BeautifulSoup(driver.page_source)

            kyobo_isbn = bs.select_one("div.tbl_row_wrap > table > tbody > tr:nth-child(1) > td")
            isbn = kyobo_isbn.text

            kyobo_name = bs.select_one("div.auto_overflow_contents > div > h1 > span")
            name = kyobo_name.text

            kyobo_genre = bs.select_one("div.intro_book > ul > li:nth-child(1) > a:nth-child(3)")
            genre = kyobo_genre.text

            kyobo_writer = bs.select_one("div.auto_overflow_contents > div > div > a:nth-child(1)")
            writer = kyobo_writer.text

            kyobo_score = bs.select_one("div.prod_review_box > div:nth-child(1) > span.review_score.feel_lucky")
            score = kyobo_score.text

            kyobo_price = bs.select_one("div.prod_price_wrap > div.prod_price_box > div > span.price > span")
            price = kyobo_price.text

            kyobo_date = bs.select_one("div.tbl_row_wrap > table > tbody > tr:nth-child(2) > td")
            date = kyobo_date.text.replace("\n","").replace(" ","")[:11]

            kyobo_list.append([isbn, name, genre, writer, score, price, date])

            time.sleep(2)    
            driver.close()

        value = str(value)
        kyobo_year = "20"+ value[:2]
        kyobo_month = value[2:]

        return make_csv(kyobo_list, kyobo_year, kyobo_month)

    except:
        pass



def make_csv(kyobo_list, kyobo_year, kyobo_month):
    
    list = ['ISBN', '책 제목', '도서 장르', '저자', '평점', '판매가격', '발행일']
    
    index_df = pd.DataFrame(columns=list)
    index_df.loc[0] = ['', '', '', '', '', '', '']
    index_df.index = [str(kyobo_year) + '년' + str(kyobo_month) + '월']
        
    book_list = pd.DataFrame(np.array(kyobo_list), columns=list)
    book_list.index = range(1, len(book_list)+1)
    book_list = pd.concat([index_df,book_list])
    
    if not os.path.exists('KyoboBestSeller.csv') :
        book_list.to_csv('KyoboBestSeller.csv', encoding='CP949', mode = 'w')
    else :
        book_list.to_csv('KyoboBestSeller.csv', encoding='CP949', mode = 'a', header=False)

    return print(f"{kyobo_year}년 {kyobo_month}월 저장완료!")



def search_day():
    for i in range(1,12):
        if i < 7:
            year = 2206
            year += i
        else:
            year = 2300
            year += (i-6)
    
        kyobo_book(year)
   
  
search_day()

2022년 07월 저장완료!
2022년 08월 저장완료!
2022년 09월 저장완료!
2022년 10월 저장완료!
2022년 11월 저장완료!
2022년 12월 저장완료!
2023년 01월 저장완료!
2023년 02월 저장완료!
2023년 03월 저장완료!
2023년 04월 저장완료!
2023년 05월 저장완료!


- request.get(url) 했을 시 출력되는 내용

In [2]:
from bs4 import BeautifulSoup
import requests

url = 'https://product.kyobobook.co.kr/bestseller/total?period=003#?page=1&per=20&period=003&ymw=202207&bsslBksClstCode=A'

res = requests.get(url)
res.encoding="utf-8"

soup = BeautifulSoup(res.text, "html.parser")
print(soup)

﻿
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<meta charset="utf-8"/>
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title> 서비스 점검 - 교보문고 </title>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi" name="viewport"/>
<meta content="" name="author">
<meta content="" name="keywords">
<meta content="" name="description"/>
<style type="text/css">
	* {padding:0;margin:0;list-style:none;}
  </style>
</meta></meta></head>
<body>
<style type="text/css">
	.errorPage_wrap {width:680px;margin:117px auto 0;}
	.errorPage_wrap .logo {font-size: 0; line-height: 0; text-align:center;}
	.errorPage_wrap .title {font-size:45px; color:#333; margin:23px 0 0 0; text-align:center;}
	.errorPage_wrap .title strong{color:#ff6600;}

	.errorPage_wrap .errorMsg01_wrap {margin:45px 0 46px;text-align:center; font-size:18px; line-height:1.4; color: